In [33]:
from typing import Dict
import tempfile
from pathlib import Path
import numpy as np
from mlflow.tracking import MlflowClient
from .minigrid import MiniGrid

FPS = 10
B, T = 5, 50#We take B dreams and put them together, We take T images of each dream

def download_artifact_npz(run_id, artifact_path) -> Dict[str, np.ndarray]:
    client = MlflowClient()
    with tempfile.TemporaryDirectory() as tmpdir:
        path = client.download_artifacts(run_id, artifact_path, tmpdir)
        with Path(path).open('rb') as f:
            data = np.load(f)
            return {k: data[k] for k in data.keys()}  # type: ignore

def encode_gif(frames, fps):
    # Copyright Danijar
    from subprocess import Popen, PIPE
    h, w, c = frames[0].shape
    # print(f"frames shape {frames[0].shape}")
    # print("min max")
    # print(np.min(frames[0]))
    # print(np.max(frames[0]))
    pxfmt = {1: 'gray', 3: 'rgb24'}[c]
    cmd = ' '.join([
        'ffmpeg -y -f rawvideo -vcodec rawvideo',
        f'-r {fps:.02f} -s {w}x{h} -pix_fmt {pxfmt} -i - -filter_complex',
        '[0:v]split[x][z];[z]palettegen[y];[x]fifo[x];[x][y]paletteuse',
        f'-r {fps:.02f} -f gif -'])
    proc = Popen(cmd.split(' '), stdin=PIPE, stdout=PIPE, stderr=PIPE)
    for image in frames:
        # print(image)
        proc.stdin.write(image.tobytes())  # type: ignore
    out, err = proc.communicate()
    if proc.returncode:
        raise IOError('\n'.join([' '.join(cmd), err.decode('utf8')]))
    del proc
    return out

def make_gif(env_name, run_id, step, fps=FPS):
    dest_path = f'figures/dream_{env_name}_{step}.gif'
    artifact = f'd2_wm_dream/{step}.npz'
    data = download_artifact_npz(run_id,artifact) 
    # print(data.keys())
    img = data['image_pred']
    # print("Img shape")
    # print(img.shape)
    # print(f"other shape {img[:B, :T].reshape(-1, 64, 64, 3).shape}")
    img = img[:B, :T].reshape((-1, 64, 64, 3))
    gif = encode_gif(img, fps)
    with Path(dest_path).open('wb') as f:
        f.write(gif)


def make_gif_episode(env_name, run_id, step, fps=FPS):
    dest_path = f'figures/episode_{env_name}_{step}.gif'
    artifact = f'episodes/0/{step}.npz'
    data = download_artifact_npz(run_id,artifact) 
    # print(data.keys())
    img = data['image_t'].transpose(3, 0, 1, 2)  # HWCT => THWC
    # print("Img shape")
    # print(img.shape)
    img = img.reshape(-1, 64, 64, 3,order='F')
    # print(f"other shape {img[:B, :T].reshape(-1, 64, 64, 3).shape}")
    # img = img[:B, :T].reshape((-1, 64, 64, 3))
    gif = encode_gif(img, fps)
    with Path(dest_path).open('wb') as f:
        f.write(gif)

def make_gif_episode_eval(env_name, run_id, step, fps=FPS):
    dest_path = f'figures/episode_{env_name}_{step}.gif'
    artifact = f'episodes_eval/0/{step}.npz'
    data = download_artifact_npz(run_id,artifact) 
    # print(data.keys())
    img = data['image_t'].transpose(3, 0, 1, 2)  # HWCT => THWC
    # print("Img shape")
    # print(img.shape)
    img = img.reshape(-1, 64, 64, 3,order='F')
    # print(f"other shape {img[:B, :T].reshape(-1, 64, 64, 3).shape}")
    # img = img[:B, :T].reshape((-1, 64, 64, 3))
    gif = encode_gif(img, fps)
    with Path(dest_path).open('wb') as f:
        f.write(gif)



def make_gif_minigrid(env_name, run_id, step, fps=FPS):
    dest_path = f'figures/dream_mapa_{env_name}_{step}.gif'
    artifact = f'd2_wm_dream/{step}.npz'
    data = download_artifact_npz(run_id, artifact)#Minigrid
    print("minigrid keys")
    print(data.keys())
    img = data['map_agent']
    print("im shape")
    print(img.shape)

    # print(f"other shape {img[:B, :T].reshape(-1, 64, 64, 3).shape}")
    img = img.reshape((-1, 8, 8))
    
    # img = img[:B, :T,:,:,:4].reshape((-1, 7, 7, 4),order='F')
    # l = len(img[:,0,0,0])
    # new_shape = (img.shape[0],img.shape[1],img.shape[2],3)
    # print(new_shape)
    # img = img[:,:,:,:3]
    # rgb = np.empty(new_shape)
    # for i in range(l):
    #     rgb[i] = rgba2rgb(img[i,:,:,:])
    # print("new image shape")
    # print(rgb.shape)
    other = np.expand_dims(img,axis =3)
    other = other*50 #Because the values are so low
    gif = encode_gif(other, fps)
    with Path(dest_path).open('wb') as f:
        f.write(gif)
def make_gif_minigrid_episode_eval(env_name, run_id, step, fps=FPS):
    dest_path = f'figures/episodes_eval_mapa_{env_name}_{step}.gif'
    artifact = f'episodes_eval/0/{step}.npz'
    data = download_artifact_npz(run_id, artifact)#Minigrid
    # print(data.keys())

    img = data['map_agent']
    print(img.shape)
    # print(f"other shape {img[:B, :T].reshape(-1, 64, 64, 3).shape}")
    # img = img[:B, :T,:,:,:4].reshape((-1, 7, 7, 4),order='F')
    # rgb = np.empty(new_shape)
    # for i in range(l):
    #     rgb[i] = rgba2rgb(img[i,:,:,:])
    other = np.expand_dims(img,axis =3)
    other = other*50 #Because the values are so low
    gif = encode_gif(other, fps)
    with Path(dest_path).open('wb') as f:
        f.write(gif)


def rgba2rgb( rgba, background=(255,255,255) ):
    row, col, ch = rgba.shape#Minigrid
    if ch == 3:
        return rgba

    assert ch == 4, 'RGBA image has 4 channels.'

    rgb = np.zeros( (row, col, 3), dtype='float32' )
    r, g, b, a = rgba[:,:,0], rgba[:,:,1], rgba[:,:,2], rgba[:,:,3]

    a = np.asarray( a, dtype='float32' ) / 255.0

    R, G, B = background

    rgb[:,:,0] = r * a + (1.0 - a) * R
    rgb[:,:,1] = g * a + (1.0 - a) * G
    rgb[:,:,2] = b * a + (1.0 - a) * B

    return np.asarray( rgb, dtype='uint8' )

ImportError: attempted relative import with no known parent package

In [ ]:
def render_map(map_, tile_size=16):
    map_ = MiniGrid.from_categorical(map_)
    # Find and remove special "agent" object
    agent_pos, agent_dir = None, None
    x, y = (map_[:, :, 0] == OBJECT_TO_IDX['agent']).nonzero()
    if len(x) > 0:
        x, y = x[0], y[0]
        agent_pos = x, y
        agent_dir = map_[x][y][2]
        map_[x][y] = np.array([1, 0, 0])  # EMPTY

    grid, vis_mask = gym_minigrid.minigrid.Grid.decode(map_)
    img = grid.render(tile_size, agent_pos=agent_pos, agent_dir=agent_dir, highlight_mask=~vis_mask)
    return img

In [31]:
#Minigrid

make_gif_minigrid('minigrid', '2fdd91da643b4b20a6f06d398f5c554f', '0001001')# Shitty dreams
make_gif_minigrid('minigrid', '342ccaea0b0b4812929cb5433bac3510', '0002001')
make_gif_minigrid('minigrid', 'bbd7ad6e65474302b9bdb60aed65835f', '0010001')



# make_gif_minigrid_episode_eval('minigrid','342ccaea0b0b4812929cb5433bac3510','ep005321_005411-0-r89-1002')
# make_gif_minigrid_episode_eval('minigrid_door','bbd7ad6e65474302b9bdb60aed65835f','ep000056_000057-0-r0-1000')

# /home/theomichel/work/pyDreamer/pydreamer-minigrid/results/baselines/atari/mlruns/0/342ccaea0b0b4812929cb5433bac3510/artifacts/episodes_eval/0/ep005321_005411-0-r89-1002.npz

/tmp/ipykernel_1286761/3892538322.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)
/tmp/ipykernel_1286761/3892538322.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)
/tmp/ipykernel_1286761/3892538322.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


minigrid keys
dict_keys(['map_agent', 'map_masked', 'agent_dir', 'action', 'reset', 'terminal', 'map', 'image', 'agent_pos', 'action_next', 'reward', 'map_seen_mask', 'map_coord', 'action_pred', 'reward_pred', 'terminal_pred', 'image_pred', 'value', 'value_target', 'value_advantage', 'value_advantage_gae', 'value_weight'])
im shape
(5, 15, 8, 8)
minigrid keys
dict_keys(['map_masked', 'map', 'reset', 'terminal', 'agent_dir', 'action', 'reward', 'action_next', 'map_agent', 'agent_pos', 'image', 'map_seen_mask', 'map_coord', 'action_pred', 'reward_pred', 'terminal_pred', 'image_pred', 'value', 'value_target', 'value_advantage', 'value_advantage_gae', 'value_weight'])
im shape
(5, 15, 8, 8)
minigrid keys
dict_keys(['action', 'map_agent', 'map', 'reward', 'map_masked', 'reset', 'agent_pos', 'image', 'action_next', 'agent_dir', 'terminal', 'map_seen_mask', 'map_coord', 'action_pred', 'reward_pred', 'terminal_pred', 'image_pred', 'value', 'value_target', 'value_advantage', 'value_advantage_ga

In [3]:
#Alien
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0000001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0001001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0005001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0010001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0201001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0301001')
# #2nd day
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0601001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0740001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '0835001')
# make_gif('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', '1185001')


#Eval
make_gif_episode_eval('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', 'ep006039_006040-0-r2640-1997')
make_gif_episode_eval('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', 'ep010009_010009-0-r2860-1270')
make_gif_episode_eval('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', 'ep011038_011039-0-r3090-1604')
make_gif_episode_eval('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', 'ep015200_015201-0-r2600-1831')
make_gif_episode_eval('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', 'ep017289_017289-1-r2390-1169')
make_gif_episode_eval('alien', 'c1a7c69b35fa4570915c6be36f57f2c9', 'ep019902_019903-1-r3970-1160')#Plays really well and goes onto the next level


/tmp/ipykernel_1286761/1269798160.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


min max
0
239


/tmp/ipykernel_1286761/1269798160.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


min max
0
242


/tmp/ipykernel_1286761/1269798160.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


...

min max
0
246


/tmp/ipykernel_1286761/1269798160.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


min max
0
247


/tmp/ipykernel_1286761/1269798160.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


min max
0
241


In [4]:
# Adventure 
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0000001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0001001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0086001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0150001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0201001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0250001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0300001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0314001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0308001')
# make_gif('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', '0643001')#Still no learning we can see the key

# /home/theomichel/work/pyDreamer/pydreamer-minigrid/results/baselines/atari/mlruns/0/261d3a26b2b842ec990a8d0a5d6111ac/artifacts/d2_wm_dream/0643001.npz
# make_gif_episode('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', 'ep000257_000257-0-r0-1001')
# make_gif_episode_eval('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', 'ep000283_000283-5-r0-1142')
# make_gif_episode_eval('adventure', '261d3a26b2b842ec990a8d0a5d6111ac', 'ep000658_000658-0-r0-1108')




In [5]:
# Montezuma

make_gif('montezuma', '599e69d178ca4f65a10423d272f9f45d', '0500001')

/tmp/ipykernel_1286761/1269798160.py:13: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id, artifact_path, tmpdir)


MlflowException: Run '599e69d178ca4f65a10423d272f9f45d' not found

In [ ]:
# Breakout

make_gif('breakout', '83e5def4975242ccbf16a3ca8f62a674', '0500001')

In [ ]:
# Space invaders

make_gif('invaders', '6d57d49ab844475cbb83b606816b01fe', '0500001')

In [ ]:
# DMC quadruped

make_gif('quadruped', 'ff6cb24c04de4e6b821bb811c855d207', '0300001')

In [ ]:
# DMLab goals small

make_gif('dmlab', '6f78cce067464e8aa4bcb6f35a1a4386', '0161001', fps=8)

In [ ]:
# MiniWorld ScavengerHunt

make_gif('scavenger', '123b575400874f5db75ac7887f4e61c0', '0900001')

In [ ]:
# make_gif('pong', '6e7cd15f26854e42a458c358d21b65c9', '0000001')
make_gif('pong', 'a4efeae409604aa4a0f8455488dae462', '0002001')
make_gif('pong', 'a4efeae409604aa4a0f8455488dae462', '0004001')
make_gif('pong', 'a4efeae409604aa4a0f8455488dae462', '0006001')

